In [1]:
import json
import pandas as pd

In [ ]:
Questions:
    - If brenda and Kegg have different sys names, which one should we use?
    - If substrates have different prefixes, how do we handle?

In [65]:
kegg_path = 'export/kegg.json'
brenda_path = 'export/brenda.json'

export_file = 'export/combined.json'

In [34]:
poster_child = '1.14.13.2'
sample_child = '2.5.1.9'

In [4]:
def read_json_data(path):
    with open(path) as json_file:
        return json.load(json_file)

In [5]:
kegg_data = read_json_data(kegg_path)

In [6]:
brenda_data = read_json_data(brenda_path)

In [59]:
combined_data = {}

In [53]:
brenda_data['2.5.1.9'].keys()

dict_keys(['SYSNAME', 'REACTIONS', 'SYNONYMS', 'SUBSTRATE', 'PRODUCT', 'PDB', 'EC_NUMBER'])

In [54]:
kegg_data['2.5.1.9'].keys()

dict_keys(['ENTRY', 'NAME', 'CLASS', 'SYSNAME', 'REACTION', 'ALL_REAC', 'SUBSTRATE', 'PRODUCT', 'COMMENT', 'HISTORY', 'REFERENCE', 'PATHWAY', 'ORTHOLOGY', 'GENES', 'DBLINKS', 'KEGG_ID', 'EC_NUMBER'])

In [60]:
for enzyme in kegg_data.values():
    ec = enzyme['EC_NUMBER']
    combined_data[ec] = {
        'SYSNAME': enzyme.get('SYSNAME', None),
        'SUBSTRATE': enzyme.get('SUBSTRATE', None), 
        'PRODUCT': enzyme.get('PRODUCT', None)
    }

In [61]:
for enzyme in brenda_data.values():
    ec = enzyme['EC_NUMBER']
    combined_enzyme = combined_data.get(ec, False)
    if combined_enzyme:
        if (enzyme['SYSNAME'] != combined_enzyme.get('SYSNAME')):
            print(f'''{ec} has two different sysnames, will use Kegg\'s
                    B: {enzyme['SYSNAME']} 
                    K: {combined_enzyme.get('SYSNAME')}''')
    
        combined_data[ec] = {
            'SYSNAME': combined_enzyme.get('SYSNAME'),
            'SUBSTRATE': sorted(set(combined_enzyme.get('SUBSTRATE',[]) + enzyme['SUBSTRATE'])),
            'PRODUCT': sorted(set(combined_enzyme.get('PRODUCT',[]) + enzyme['PRODUCT']))
        }
    else:
        combined_data[ec] = {
            'SYSNAME': enzyme['SYSNAME'],
            'SUBSTRATE': enzyme['SUBSTRATE'],
            'PRODUCT': enzyme['PRODUCT']
        }

1.13.11.79 has two different sysnames, will use Kegg's
                    B: FMNH2 oxidoreductase (5,6-dimethylbenzimidazole forming) 
                    K: FMNH2 oxidoreductase (5,6-dimethylbenzimidazole-forming)
1.14.13.113 has two different sysnames, will use Kegg's
                    B: urate,NADH:oxygen oxidoreductase (5-hydroxyisourate-forming) 
                    K: urate,NADH:oxygen oxidoreductase (5-hydroxyisourate forming)
1.14.14.8 has two different sysnames, will use Kegg's
                    B: anthranilate,FAD:oxygen oxidoreductase (3-hydroxylating) 
                    K: anthranilate,FADH2:oxygen oxidoreductase (3-hydroxylating)
1.14.14.9 has two different sysnames, will use Kegg's
                    B: 4-hydroxyphenylacetate,FAD:oxygen oxidoreductase (3-hydroxylating) 
                    K: 4-hydroxyphenylacetate,FADH2:oxygen oxidoreductase (3-hydroxylating)
1.14.19.3 has two different sysnames, will use Kegg's
                    B: linoleoyl-CoA,hydrogen-donor

In [66]:
# Writing out the results to the file
with open(export_file, 'w') as outfile:
    json.dump(combined_data, outfile)

In [18]:
kegg_ecs = [entree['EC'].replace('ec:','') for entree in kegg_data.values()]
brenda_ecs = [entree['SYSNAME'][0]['ecNumber'] for entree in brenda_data.values()]

In [43]:
kegg_sys_names = [k for k in kegg_data.keys()]
brenda_sys_names = [entree['SYSNAME'][0]['systematicName'] for entree in brenda_data.values()]

In [42]:
(list(brenda_data.values())[0]['SYSNAME'])

[{'systematicName': 'alcohol:NAD+ oxidoreductase', 'ecNumber': '1.1.1.1'}]

In [33]:
len(set(brenda_ecs)&set(kegg_ecs))

74

In [34]:
len(kegg_ecs)

78

In [90]:
len(set(brenda_ecs)|set(kegg_ecs))

780

In [88]:
len(set(kegg_sys_names)&set(brenda_sys_names))

16

In [50]:
sysnames_not_in_brenda = set(kegg_sys_names) - (set(kegg_sys_names)&set(brenda_sys_names))

In [51]:
len(sysnames_not_in_brenda)

16

In [62]:
sorted_sysnames_not_in_brenda = sorted(list(sysnames_not_in_brenda))

In [69]:
one = sorted_sysnames_not_in_brenda[0]
one_ec = kegg_data[one]['EC'].replace('ec:','')
one_ec

'6.2.1.42'

In [74]:
brenda_data[one_ec]['SYSNAME'][0]['systematicName']

'3-oxocholest-4-en-26-oate:CoA ligase (AMP-forming)'

In [72]:
one

'(25S)-3-oxocholest-4-en-26-oate:CoA ligase (AMP-forming)'

In [ ]:
final_db = {}

# joinining the lists
for enzyme_ec in brenda_data:
    final_db[enzyme_ec]['NAME'] = {syn['synonyms'] for syn in brenda_data[poster_child]['NAME']}
    

In [93]:
{syn['synonyms'] for syn in brenda_data[poster_child]['NAME']}

{'4-HBA 3-hydroxylase',
 '4-HBA 3-monooxygenase',
 '4-HBA-3-hydroxylase',
 '4-HBMO',
 '4-hydroxybenzoate 3-hydroxylase',
 '4-hydroxybenzoate 3-monooxygenase',
 '4-hydroxybenzoate hydroxylase',
 '4-hydroxybenzoate monooxygenase',
 '4-hydroxybenzoic hydroxylase',
 '4HBA 3-hydroxylase',
 'BxeA2040',
 'HBH',
 'MobA',
 'PHBAD',
 'PHBH',
 'PHBHase',
 'POHBase',
 'PobA',
 'm-hydroxybenzoate hydroxylase',
 'ncgl1032',
 'oxygenase, 4-hydroxybenzoate 3-mono-',
 'p-hydroxybenzoate hydroxylase',
 'p-hydroxybenzoate-3-hydroxylase',
 'p-hydroxybenzoic acid hydrolase',
 'p-hydroxybenzoic acid hydroxylase',
 'p-hydroxybenzoic hydroxylase',
 'para-hydroxybenzoate hydroxylase',
 'pobACg'}

In [95]:
[syn for syn in brenda_data[poster_child]['SUBSTRATE']]

[{'reactionPartners': '2,3-dihydroxybenzoate + NADPH + O2 = ? + NADP+ + H2O',
  'substrate': '2,3-dihydroxybenzoate',
  'organism': 'Comamonas testosteroni',
  'ecNumber': '1.14.13.2',
  'ligandStructureId': 1367},
 {'reactionPartners': '2,3-dihydroxybenzoate + NADPH + O2 = ? + NADP+ + H2O',
  'substrate': '2,3-dihydroxybenzoate',
  'organism': 'Comamonas testosteroni GZ39',
  'ecNumber': '1.14.13.2',
  'ligandStructureId': 1367},
 {'reactionPartners': '2,4-dihydroxybenzoate + NADPH + O2 = 2,3,4-trihydroxybenzoate + 2,4,5-trihydroxybenzoate + NADP+ + H2O',
  'substrate': '2,4-dihydroxybenzoate',
  'organism': 'Pseudomonas fluorescens',
  'ecNumber': '1.14.13.2',
  'ligandStructureId': 1909},
 {'reactionPartners': '2,4-dihydroxybenzoate + NADPH + O2 = 2,3,4-trihydroxybenzoate + 2,4,5-trihydroxybenzoate + NADP+ + H2O',
  'substrate': '2,4-dihydroxybenzoate',
  'organism': 'Pseudomonas aeruginosa',
  'ecNumber': '1.14.13.2',
  'ligandStructureId': 1909},
 {'reactionPartners': '2,4-dihydro

In [86]:
brenda_data[poster_child]['NAME']

[{'commentary': None,
  'synonyms': '4-HBA 3-hydroxylase',
  'literature': [742538],
  'organism': 'Alcaligenes sp. PPH',
  'ecNumber': '1.14.13.2'},
 {'commentary': None,
  'synonyms': '4-HBA 3-hydroxylase',
  'literature': [742538],
  'organism': 'Pseudomonas sp. PPD',
  'ecNumber': '1.14.13.2'},
 {'commentary': None,
  'synonyms': '4-HBA 3-hydroxylase',
  'literature': [743802],
  'organism': 'Xanthomonas campestris',
  'ecNumber': '1.14.13.2'},
 {'commentary': None,
  'synonyms': '4-HBA 3-hydroxylase',
  'literature': [0],
  'organism': 'Xanthomonas campestris ATCC 33913',
  'ecNumber': '1.14.13.2'},
 {'commentary': None,
  'synonyms': '4-HBA 3-monooxygenase',
  'literature': [726322],
  'organism': 'Paraburkholderia xenovorans',
  'ecNumber': '1.14.13.2'},
 {'commentary': None,
  'synonyms': '4-HBA 3-monooxygenase',
  'literature': [743802],
  'organism': 'Xanthomonas campestris',
  'ecNumber': '1.14.13.2'},
 {'commentary': None,
  'synonyms': '4-HBA 3-monooxygenase',
  'literatur